In [1]:
require(Lightbulb)

Loading required package: Lightbulb
Loading required package: data.table
Loading required package: matrixStats
Loading required package: Matrix
Loading required package: gplots

Attaching package: ‘gplots’

The following object is masked from ‘package:stats’:

    lowess

Loading required package: ggplot2
Loading required package: Seurat
Loading required package: cowplot

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave


Attaching package: ‘Lightbulb’

The following objects are masked from ‘package:matrixStats’:

    colMaxs, colMins, colSds, rowMaxs, rowMins, rowSds



## loading data

In [2]:
Exp_Seurat=readRDS("Lightbulb_example_ExpSeurat.rds")

### Super cell creation

Exp_Seurat is a Seurat object. Seurat object stores the log2(TPM) matrix in @data slot. Our super cell function can take a seurat object as input or a cell-gene matrix as input. For details about generating your own Seurat object, please refer to Seurat manual (https://satijalab.org/seurat/get_started.html)

In [3]:
Exp_Seurat@data[1:5,1:5]

5 x 5 sparse Matrix of class "dgCMatrix"
        CAGTCCTTCCAAGTAC-1 TAAGTGCTCTCTGAGA-1 CGCTTCATCTTACCTA-1
Mrpl15            .                  8.470585                  .
Lypla1            .                  8.470585                  .
Tcea1             8.633139           8.470585                  .
Atp6v1h           .                  .                         .
Rb1cc1            .                  8.470585                  .
        TCTTCGGAGTTTAGGA-1 AATCCAGCATAGACTC-1
Mrpl15                   .                  .
Lypla1                   .                  .
Tcea1                    .                  .
Atp6v1h                  .                  .
Rb1cc1                   .                  .

In [4]:
Exp_Seurat@meta.data[1:10,]

,nGene,nUMI,orig.ident,batch,res.1,tissue,timepoint,test,res.3
CAGTCCTTCCAAGTAC-1,945,2525,SeuratProject,1,8,Spleen,d0,0,0
TAAGTGCTCTCTGAGA-1,891,2827,SeuratProject,1,8,Spleen,d0,0,0
CGCTTCATCTTACCTA-1,981,3012,SeuratProject,1,8,Spleen,d0,0,0
TCTTCGGAGTTTAGGA-1,744,2212,SeuratProject,1,8,Spleen,d0,0,0
AATCCAGCATAGACTC-1,890,2455,SeuratProject,1,8,Spleen,d0,0,0
CTTAGGAGTGTTAAGA-1,680,1628,SeuratProject,1,8,Spleen,d0,0,0
TGACTAGTCCTTTCGG-1,996,3462,SeuratProject,1,8,Spleen,d0,0,0
GACACGCGTACCGTAT-1,717,2552,SeuratProject,1,8,Spleen,d0,0,0
CTAATGGCACTGTGTA-1,716,1631,SeuratProject,1,8,Spleen,d0,0,0
TTAGGCATCCTGCCAT-1,856,2483,SeuratProject,1,8,Spleen,d0,0,0


#### using seurat object as input
When taking Seurat object as input, the calculation is based on @scale.data slot. We need to make sure that @scale.data is addible. In this example, we simply put the TPM matrix in the @scale.data slot.

*k_merge* an integer indicates the number of cells we want to merge to create a super-cell.  
*n* an integer of the number of super-cell we want to get.  
*sampling_ref* a vector of group_id for each cell. The sampling process will try to make sure the group_id distribution remains the same after sampling.  

In [5]:
Exp_Seurat@scale.data=2^Exp_Seurat@data-1

#calculate super cell
supercell_mat=Super_cell_creation(Exp_Seurat,k_merge = 50,n=6000,sampling_ref = Exp_Seurat@meta.data$batch)
supercell_mat=log2(supercell_mat+1)

Seurat object detected as input
MNN done: time consumed: 0 hr 1 min 51.37 s
merging finished: time consumed: 0 hr 4 min 54.86 s


#### using cell gene matrix  as input
The cell gene matrix is a matrix which each row represents a cell and each column is a gene. We need to make sure the value of cell-gene matrix is addible. In this example, we simply use the TPM matrix.

In [18]:
TPM_matrix=as.matrix(t(2^Exp_Seurat@data-1))

#calculate super cell
supercell_mat=Super_cell_creation(TPM_matrix,k_merge = 50,n=6000,sampling_ref = Exp_Seurat@meta.data$batch)
supercell_mat=log2(supercell_mat+1)

Assuming input is cell-gene matrix
MNN done: time consumed: 0 hr 1 min 34.67 s
merging finished: time consumed: 0 hr 5 min 26.77 s


The row name of the super-cell matrix indicate the position of the cell center in the original data. For example in the following super-cell matrix, the first super-cell is centered on cell 193 in orginal data (row 193 in TPM_matrix or column 193 in Exp_Seurat@scale.data).

In [8]:
supercell_mat[1:5,1:5]

,Mrpl15,Lypla1,Tcea1,Atp6v1h,Rb1cc1
193,5.458230,5.265078,5.926450,4.828982,5.057072
276,5.754755,5.342182,4.219048,5.058700,4.709162
1234,6.467162,5.142626,5.424017,5.212062,4.988101
651,6.353871,4.017780,5.602713,4.489083,3.023346
495,5.606721,5.507785,5.874736,4.482352,3.114476
